# Homework 3

## Data Preparation

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv(r"C:\Users\mab03\Desktop\ML-Zoomcamp\03-classification\course_lead_scoring.csv")
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [6]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [9]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [13]:
categorical = df.select_dtypes(include=['object']).columns.tolist()
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [14]:
numerical = df.select_dtypes(exclude=['object']).columns.tolist()
numerical 

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [15]:
# Replace categorical features with 'NA'
for col in categorical:
    df[col] = df[col].fillna('NA')

In [16]:
df.isnull().sum()

lead_source                   0
industry                      0
number_of_courses_viewed      0
annual_income               181
employment_status             0
location                      0
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [17]:
# Replace numerical features with 0.0
for col in numerical:
    df[col] = df[col].fillna(0.0)

In [19]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

## Question 1

In [20]:
# Most frequent observation for 'industry'
df.industry.mode()

0    retail
Name: industry, dtype: object

## Question 2

In [24]:
corr_matrix = df[numerical].corr()
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


In [33]:
# Split the data
from sklearn.model_selection import train_test_split

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

y_train = df_train['converted'].values
y_val = df_val['converted'].values
y_test = df_test['converted'].values


del df_train['converted']
del df_val['converted']
del df_test['converted']

print(len(df_train), len(df_val), len(df_test))

876 293 293


## Question 3

In [ ]:
# Calculate the Mutual Information score between converted (target) and other categorical variables
from sklearn.metrics import mutual_info_score
def calculate_mi(series):
    return round(mutual_info_score(series, y_train),2)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')
df_mi

,MI
lead_source,0.04
industry,0.01
employment_status,0.01
location,0.00


## Question 4

In [54]:
# Train a logistic regression
# numerical without target
numerical = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',]
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)

In [ ]:
dv.feature_names_

['annual_income',
 'employment_status=NA',
 'employment_status=employed',
 'employment_status=self_employed',
 'employment_status=student',
 'employment_status=unemployed',
 'industry=NA',
 'industry=education',
 'industry=finance',
 'industry=healthcare',
 'industry=manufacturing',
 'industry=other',
 'industry=retail',
 'industry=technology',
 'interaction_count',
 'lead_score',
 'lead_source=NA',
 'lead_source=events',
 'lead_source=organic_search',
 'lead_source=paid_ads',
 'lead_source=referral',
 'lead_source=social_media',
 'location=NA',
 'location=africa',
 'location=asia',
 'location=australia',
 'location=europe',
 'location=middle_east',
 'location=north_america',
 'location=south_america',
 'number_of_courses_viewed']

In [55]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [56]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [57]:
y_pred = model.predict_proba(X_val)[:, 1]

In [58]:
from sklearn.metrics import accuracy_score
baseline_accuracy = accuracy_score(y_val, y_pred >= 0.5)
round(accuracy_score(y_val, y_pred >= 0.5),2)

0.7

# Question 5

In [52]:
# Exclude each feature and record accuracy for each model
numerical = ['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',]
all_columns = categorical + numerical
without_feature_accuracy = dict()

for col in all_columns:
    train_cols = df_train.loc[:, df_train.columns != col].columns
    train_dict = df_train[train_cols].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    dv.fit(train_dict)
    X_train = dv.transform(train_dict)
    
    # Fit the model
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train,y_train)
    
    # Validation performance
    val_dict = df_val[train_cols].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    
    y_pred = model.predict_proba(X_val)[:, 1]
    without_feature_accuracy[col] = accuracy_score(y_val, y_pred >= 0.5)
    

In [53]:
without_feature_accuracy

{'lead_source': 0.7030716723549488,
 'industry': 0.6996587030716723,
 'employment_status': 0.6962457337883959,
 'location': 0.7098976109215017,
 'number_of_courses_viewed': 0.5563139931740614,
 'annual_income': 0.8532423208191127,
 'interaction_count': 0.5563139931740614,
 'lead_score': 0.7064846416382252}

In [68]:
diffs = [baseline_accuracy - acc for acc in list(without_feature_accuracy.values())]
diffs

[-0.0034129692832765013,
 0.0,
 0.0034129692832763903,
 -0.010238907849829393,
 0.14334470989761094,
 -0.15358361774744034,
 0.14334470989761094,
 -0.0068259385665528916]

# Question 6

In [76]:
# Regularized regression
train_dict = df_train[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
X_train = dv.transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

c_vals = [0.01, 0.1, 1, 10, 100]
for c in c_vals:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train,y_train)

    y_pred = model.predict_proba(X_val)[:, 1]
    print(round(accuracy_score(y_val, y_pred >= 0.5),3))

0.7
0.7
0.7
0.7
0.7
